In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib as jb


In [2]:
excel_file = r"D:\Greenhouse Gas Emission.xlsx"
years = range(2010,2017)

In [3]:
years[0]

2010

In [4]:
GHG = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Commodity')
GHG.head()

,Commodity Code,Commodity Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",carbon dioxide,"kg/2018 USD, purchaser price",0.398,0.073,0.470,NaN,4,3,1,4,1
1,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1
2,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",nitrous oxide,"kg/2018 USD, purchaser price",0.002,0.000,0.002,NaN,4,3,1,4,1
3,1111A0,"Fresh soybeans, canola, flaxseeds, and other o...",other GHGs,"kg CO2e/2018 USD, purchaser price",0.002,0.000,0.002,NaN,3,3,1,3,1
4,1111B0,"Fresh wheat, corn, rice, and other grains",carbon dioxide,"kg/2018 USD, purchaser price",0.659,0.081,0.740,NaN,4,3,1,4,1


In [5]:
GHG = pd.read_excel(excel_file, sheet_name=f'{years[0]}_Detail_Industry')
GHG.head()

,Industry Code,Industry Name,Substance,Unit,Supply Chain Emission Factors without Margins,Margins of Supply Chain Emission Factors,Supply Chain Emission Factors with Margins,Unnamed: 7,DQ ReliabilityScore of Factors without Margins,DQ TemporalCorrelation of Factors without Margins,DQ GeographicalCorrelation of Factors without Margins,DQ TechnologicalCorrelation of Factors without Margins,DQ DataCollection of Factors without Margins
0,1111A0,Oilseed farming,carbon dioxide,"kg/2018 USD, purchaser price",0.414,0.073,0.487,NaN,4,3,1,4,1
1,1111A0,Oilseed farming,methane,"kg/2018 USD, purchaser price",0.001,0.001,0.002,NaN,4,3,1,1,1
2,1111A0,Oilseed farming,nitrous oxide,"kg/2018 USD, purchaser price",0.002,0.000,0.002,NaN,4,3,1,4,1
3,1111A0,Oilseed farming,other GHGs,"kg CO2e/2018 USD, purchaser price",0.002,0.000,0.002,NaN,3,3,1,3,1
4,1111B0,Grain farming,carbon dioxide,"kg/2018 USD, purchaser price",0.680,0.082,0.762,NaN,4,3,1,4,1


In [6]:
all_data = []

for year in years:
    try:
        sheet_com = f'{year}_Detail_Commodity'
        sheet_ind = f'{year}_Detail_Industry'

        # Check if both sheets exist
        if sheet_com in excel_file.sheet_names and sheet_ind in excel_file.sheet_names:
            df_com = pd.read_excel(excel_file, sheet_name=sheet_com)
            df_ind = pd.read_excel(excel_file, sheet_name=sheet_ind)

            df_com['Source'] = 'Commodity'
            df_ind['Source'] = 'Industry'
            df_com['Year'] = df_ind['Year'] = year

            df_com.columns = df_com.columns.str.strip()
            df_ind.columns = df_ind.columns.str.strip()

            df_com.rename(columns={
                'Commodity Code': 'Code',
                'Commodity Name': 'Name'
            }, inplace=True)

            df_ind.rename(columns={
                'Industry Code': 'Code',
                'Industry Name': 'Name'
            }, inplace=True)

            all_data.append(pd.concat([df_com, df_ind], ignore_index=True))

        else:
            print(f"Sheets for year {year} not found.")

    except Exception as e:
        print(f"Error processing year {year}: {e}")


Error processing year 2010: 'str' object has no attribute 'sheet_names'
Error processing year 2011: 'str' object has no attribute 'sheet_names'
Error processing year 2012: 'str' object has no attribute 'sheet_names'
Error processing year 2013: 'str' object has no attribute 'sheet_names'
Error processing year 2014: 'str' object has no attribute 'sheet_names'
Error processing year 2015: 'str' object has no attribute 'sheet_names'
Error processing year 2016: 'str' object has no attribute 'sheet_names'


In [7]:
all_data[3]

IndexError: list index out of range

In [ ]:
df = pd.concat(all_data, ignore_index=True)
df.head()

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.isnull().sum()